In [ ]:
# download the NYC Flights dataset to our workspace
import urllib

print("- Downloading NYC Flights dataset... ", end='', flush=True)
url = "https://storage.googleapis.com/dask-tutorial-data/nycflights.tar.gz"
filename, headers = urllib.request.urlretrieve(url, '../data/csv/nycflights.tar.gz')
print("Done!", flush=True)

- Downloading Fine Foods dataset... Done!


In [ ]:
import tarfile

# extract the .csv files from the tar file
with tarfile.open(filename, mode='r:gz') as flights:
            flights.extractall('../data/csv/')

../data/raw/finefoods.txt.gz


AttributeError: 'GzipFile' object has no attribute 'extractall'

In [9]:
import os
import dask.dataframe as dd

df = dd.read_csv(os.path.join('../data/csv/', 'nycflights/', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]})

df

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
npartitions=10,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float64,int64,float64,int64,object,int64,float64,float64,int64,float64,float64,float64,object,object,float64,float64,float64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
df_train = df[['CRSDepTime',  'CRSArrTime', 'Cancelled']]
df_train.head()

,CRSDepTime,CRSArrTime,Cancelled
0,1540,1701,0
1,1540,1701,0
2,1540,1701,0
3,1540,1701,0
4,1540,1701,0


In [13]:
df_train.shape

(Delayed('int-fe2b1ecd-f4a5-4da7-b921-3049a4dd23d8'), 3)

In [14]:
df_train.isnull().sum().compute()

CRSDepTime    0
CRSArrTime    0
Cancelled     0
dtype: int64

In [32]:
#from keras.models import Sequential
#from keras.layers import Dense

# Build a simple Keras Model
#model = Sequential()
#model.add(Dense(20, input_dim=df_train.shape[1], activation='relu'))
#model.add(Dense(1,activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='sgd',)

import torch.nn as nn
import torch.nn.functional as F
from dask.diagnostics import ProgressBar
from dask_ml.wrappers import Incremental

class ShallowNet(nn.Module):
    def __init__(self, n_features=5):
        super().__init__()
        self.layer1 = nn.Linear(n_features, 1)

    def forward(self, x):
        return F.relu(self.layer1(x))
    
# from skorch import NeuralNetRegressor
import torch.optim as optim

# sn = ShallowNet()
niceties = {
    "callbacks": False,
    "warm_start": False,
    "train_split": None,
    "max_epochs": 1,
}

model = ShallowNet(
    
    optimizer=optim.SGD,
    optimizer__lr=0.1,
    optimizer__momentum=0.9,
    batch_size=64,
    **niceties,
)


parallel_shallowNet = Incremental(model)







TypeError: __init__() got an unexpected keyword argument 'optimizer'

In [16]:
# generate batches of data from our dask dataframe
def dask_data_generator(df, fraction=0.01):
  while True:
    batch = df.sample(frac=fraction)
    X = batch.iloc[:, :-1]
    y = batch.iloc[:, -1]
    yield (X.compute(), y.compute())

In [27]:
# never run out-of-memory while training
parallel_shallowNet.fit(df_train)


TypeError: Cannot clone object 'ShallowNet(
  (layer1): Linear(in_features=5, out_features=1, bias=True)
)' (type <class '__main__.ShallowNet'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.